In [1]:
data_var = '2023-12-13'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8256,2023-12-13,Noruega Blno,14:30,Froya,Fyllingen,5.72,1.12,167.5,1.85,1.85,11.5,2.00,1.72,AZieT8ol,0.174825,0.892857,0.540541,0.540541,0.067682,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,77,83,1.40,1.09,0.000,0.000,0.951079,0.000000,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
8257,2023-12-13,Libano Divisão 1,16:45,Al-Riyadi,Beirut Club,1.39,2.84,164.5,1.81,1.85,-7.5,2.09,1.66,WQqD70Oe,0.719424,0.352113,0.552486,0.540541,0.071537,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,91,1.07,1.40,0.000,0.000,0.484778,0.015456,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
8258,2023-12-13,Lituânia Moteru Lyga Feminina,12:00,Klaipedos Neptunas F,Siauliai F,1.25,3.69,144.5,1.85,1.85,-8.5,2.01,1.72,4zbPqiTs,0.800000,0.271003,0.540541,0.540541,0.071003,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.698518,0.000000,0.109952,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
8259,2023-12-13,Noruega Blno,15:00,Centrum Tigers,Ammerud,1.24,3.77,152.5,1.84,1.86,-10.5,2.04,1.69,lQjaSS0f,0.806452,0.265252,0.543478,0.537634,0.071704,149.596,28.454496,0.190209,1.8,1.643168,0.912871,193.20,2.048,0.528413,0.258014,-14.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,69,81,2.80,4.30,0.000,0.000,0.714164,0.007644,0.132701,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
8260,2023-12-13,Sérvia Primeira Liga,12:30,Mladost Zemun,Sloga,1.44,2.65,166.5,1.85,1.85,-6.5,2.09,1.66,hAyKeRSA,0.694444,0.377358,0.540541,0.540541,0.071803,194.810,113.587128,0.583066,0.6,1.341641,2.236068,148.61,2.452,1.136407,0.463461,-32.0,289.752,248.070735,0.856148,0.6,1.341641,2.236068,115.50,3.390,2.349926,0.693193,-17.0,77,70,1.93,1.65,0.000,0.000,0.418386,0.000000,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8360,2023-12-13,China Cba,08:35,Shanghai,Guangzhou,1.63,2.22,196.5,1.81,1.91,-5.5,1.95,1.75,l0BtVdjG,0.613497,0.450450,0.552486,0.523560,0.063947,209.606,117.218964,0.559235,2.4,1.341641,0.559017,134.47,2.096,1.279680,0.610534,5.0,151.336,43.261547,0.285864,1.8,1.643168,0.912871,210.00,1.484,0.362119,0.244015,34.0,119,105,1.13,2.00,195.538,243.170,0.216724,0.038016,0.076444,0.64,0.128,4.921875,0.0,0.0,0.0,2.93,0.586,2.081911,0.0,0.0,0.0
8361,2023-12-13,China Cba,08:35,Ningbo Rockets,Shandong,3.27,1.32,197.5,1.86,1.88,6.5,2.01,1.69,23FpUG5M,0.305810,0.757576,0.537634,0.531915,0.063386,646.956,241.992412,0.374048,0.0,0.000000,NaN,314.00,7.590,3.422660,0.450943,-77.0,287.568,166.154594,0.577792,1.2,1.643168,1.369306,205.00,2.738,1.785797,0.652227,4.0,100,100,3.14,2.05,946.296,251.702,0.600810,0.007563,0.122310,-2.53,-0.506,-4.486166,0.0,0.0,0.0,-5.00,-1.000,-0.320000,0

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V1 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8326,21:30,Eua Nba,Toronto Raptors,Atlanta Hawks,1.75,Back Home


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
8263,12:00,Sérvia Primeira Liga,Dynamic,Sloboda,1.66,Back Home
8264,14:00,Israel Superliga,H. Haifa,Galil Elyon,1.61,Back Home
8266,12:00,Geórgia Superliga,Kutaisi,TSU,1.99,Back Home
8269,05:00,Filipinas Copa Commissioners,Terrafirma Dyip,Converge FiberXers,2.17,Back Home
8275,16:00,Europa Euroliga Feminina,ESB Villeneuve F,Perfumerias Avenida F,1.95,Back Home
8282,15:00,Sérvia Primeira Liga,Metalac,Radnicki,2.29,Back Home
8284,22:30,Eua Ncaa,La Lafayette,Eastern Kentucky,1.76,Back Home
8288,22:00,Eua Ncaa,Louisville,Arkansas State,1.61,Back Home
8297,20:30,Eua Ncaa,Stony Brook,Norfolk State,1.77,Back Home
8302,13:00,Catar Liga De Basquete Do Catar,Al Wakrah,Al Sadd,1.72,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
8264,14:00,Israel Superliga,H. Haifa,Galil Elyon,1.87,Back Home
8346,16:30,Europa Liga Dos Campeões,Benfica,Galatasaray,1.93,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8288,22:00,Eua Ncaa,Louisville,Arkansas State,1.61,Back Home
8297,20:30,Eua Ncaa,Stony Brook,Norfolk State,1.77,Back Home
8307,21:00,Eua Ncaa,Dartmouth,Boston University,1.69,Back Home
8330,00:30,Eua Nba,Los Angeles Clippers,Sacramento Kings,1.67,Back Home
8333,20:00,Brasil Nbb,Pinheiros,R10 Score Vasco da Gama,2.20,Back Home
8360,08:35,China Cba,Shanghai,Guangzhou,1.63,Back Home
